In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import category_encoders as ce

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
import random
import xgboost as xgb

In [6]:
datos = pd.read_csv('events_up_to_01062018.csv')
tabla_completa = datos.copy()
res = pd.read_csv('labels_training_set.csv');

C:\Users\lucia\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
tabla_completa['aux']=1
#Voy a contar la cantidad para cada tipo de evento de cada persona
num_eventos = tabla_completa.groupby(['person','event'])['aux'].sum()

num_eventos = num_eventos.unstack().reset_index().fillna(0)
num_eventos = num_eventos.set_index('person')
num_eventos['total_eventos'] = num_eventos.sum(axis=1)
#normalizo los numeros obtenidos según la cantidad de eventos totales de esa persona
num_eventos_norm = num_eventos.loc[:,"ad campaign hit":"visited site"].div(num_eventos["total_eventos"], axis=0)
num_eventos_norm = num_eventos_norm.reset_index()
num_eventos_norm.head()

tabla_completa['timestamp'] = pd.to_datetime(tabla_completa['timestamp'])
#tabla_completa['timestamp'].max() -tabla_completa['timestamp'].min()

time_for_user = tabla_completa.groupby(['person'])['timestamp'].apply(lambda x: x.max() - x.min())
time_for_user = time_for_user.reset_index()
time_for_user['total_horas'] = time_for_user['timestamp'].dt.total_seconds()/3600
time_for_user = time_for_user.drop(['timestamp'],axis='columns')

max_time = tabla_completa.groupby(['person'])['timestamp'].apply(lambda x: x.max())
max_time = max_time.reset_index()
max_time['max'] = max_time['timestamp'].dt.month
max_time = max_time.drop(['timestamp'],axis='columns')

time_for_user = pd.merge(max_time,time_for_user,on='person',how='inner')

min_time = tabla_completa.groupby(['person'])['timestamp'].apply(lambda x: x.min())
min_time = min_time.reset_index()
min_time['min'] = min_time['timestamp'].dt.month
min_time = min_time.drop(['timestamp'],axis='columns')

time_for_user = pd.merge(min_time,time_for_user,on='person',how='inner')

num_eventos_norm = num_eventos_norm.reset_index()
num_eventos = num_eventos.reset_index()
total_eventos = num_eventos
#total_eventos = pd.merge(num_eventos,num_eventos_norm, on='person',how='inner')
total_eventos = pd.merge(total_eventos,time_for_user,on='person',how='inner')

In [8]:
total_eventos.head()

,person,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site,total_eventos,min,max,total_horas
0,0008ed71,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,6.0,5,5,4.013889
1,00091926,15.0,25.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,372.0,34.0,448.0,5,5,669.726111
2,00091a7a,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,10.0,3,3,0.096389
3,000ba417,1.0,24.0,6.0,1.0,14.0,0.0,1.0,0.0,0.0,153.0,6.0,206.0,5,5,217.960278
4,000c79fe,1.0,0.0,1.0,0.0,1.0,0.0,1.0,9.0,0.0,3.0,1.0,17.0,5,5,0.172222


In [10]:
total_eventos.shape

(38829, 16)

In [16]:
cat_df_flights_onehot = datos.copy()
cat_df_flights_onehot = cat_df_flights_onehot[["person","storage","condition","new_vs_returning","search_engine","channel"]]
cat_df_flights_onehot = pd.get_dummies(cat_df_flights_onehot, columns=['storage','condition','new_vs_returning','search_engine','channel'])
cat_df_flights_onehot.head()

,person,storage_128GB,storage_16GB,storage_256GB,storage_32GB,storage_4GB,storage_512MB,storage_64GB,storage_8GB,condition_Bom,...,search_engine_Bing,search_engine_Google,search_engine_Yahoo,channel_Direct,channel_Email,channel_Organic,channel_Paid,channel_Referral,channel_Social,channel_Unknown
0,4886f805,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ad93850f,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0297fc1e,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2d681dd8,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,cccea85e,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
cat_df_flights_onehot = cat_df_flights_onehot.groupby("person").sum()

In [22]:
cat_df_flights_onehot = cat_df_flights_onehot.reset_index()
cat_df_flights_onehot.head()

,person,storage_128GB,storage_16GB,storage_256GB,storage_32GB,storage_4GB,storage_512MB,storage_64GB,storage_8GB,condition_Bom,...,search_engine_Bing,search_engine_Google,search_engine_Yahoo,channel_Direct,channel_Email,channel_Organic,channel_Paid,channel_Referral,channel_Social,channel_Unknown
0,0008ed71,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
1,00091926,48.0,104.0,10.0,132.0,0.0,0.0,80.0,0.0,102.0,...,0.0,0.0,0.0,25.0,0.0,0.0,8.0,1.0,0.0,0.0
2,00091a7a,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,000ba417,0.0,115.0,1.0,20.0,1.0,0.0,1.0,22.0,115.0,...,0.0,1.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0
4,000c79fe,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [93]:
tabla = pd.merge(cat_df_flights_onehot, total_eventos, on = "person", how = "inner")
tabla.head()

,person,storage_128GB,storage_16GB,storage_256GB,storage_32GB,storage_4GB,storage_512MB,storage_64GB,storage_8GB,condition_Bom,...,lead,search engine hit,searched products,staticpage,viewed product,visited site,total_eventos,min,max,total_horas
0,0008ed71,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,2.0,6.0,5,5,4.013889
1,00091926,48.0,104.0,10.0,132.0,0.0,0.0,80.0,0.0,102.0,...,0.0,0.0,0.0,0.0,372.0,34.0,448.0,5,5,669.726111
2,00091a7a,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,3.0,1.0,10.0,3,3,0.096389
3,000ba417,0.0,115.0,1.0,20.0,1.0,0.0,1.0,22.0,115.0,...,0.0,1.0,0.0,0.0,153.0,6.0,206.0,5,5,217.960278
4,000c79fe,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,1.0,9.0,0.0,3.0,1.0,17.0,5,5,0.172222


In [52]:
tabla.shape

(38829, 42)

In [87]:
tabla_labels = pd.merge(res, tabla, on = "person", how = "inner")
tabla_labels.head()

,person,label,storage_128GB,storage_16GB,storage_256GB,storage_32GB,storage_4GB,storage_512MB,storage_64GB,storage_8GB,...,lead,search engine hit,searched products,staticpage,viewed product,visited site,total_eventos,min,max,total_horas
0,0566e9c1,0,0.0,9.0,0.0,9.0,0.0,0.0,0.0,7.0,...,0.0,1.0,0.0,1.0,23.0,17.0,68.0,5,5,214.616111
1,6ec7ee77,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2.0,5,5,0.000000
2,abe7a2fb,0,0.0,11.0,0.0,10.0,0.0,1.0,10.0,0.0,...,0.0,4.0,6.0,0.0,31.0,22.0,96.0,2,5,2593.164444
3,34728364,0,0.0,15.0,0.0,8.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,24.0,4.0,37.0,5,5,206.263611
4,87ed62de,0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,9.0,1.0,17.0,5,5,0.391389


In [90]:
tabla_labels.shape

(19414, 43)

In [32]:
y = tabla_labels["label"]
x = tabla_labels.drop(["label", "person"],axis="columns")

In [33]:
datos_train, datos_test, cat_train, cat_test = train_test_split(x, y, test_size = 0.3, random_state = 0)

In [34]:
clf = RandomForestClassifier(n_estimators=10000, random_state=0)

In [36]:
model = clf.fit(datos_train,cat_train)

In [37]:
model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10000, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [46]:
#Veo la importancia de cada variable
from tabulate import tabulate
headers = ["name", "score"]
values = sorted(zip(datos_train.columns, model.feature_importances_), key=lambda x: x[1] * -1)
print(tabulate(values, headers, tablefmt="plain"))

name                                score
total_horas                   0.0862768
checkout                      0.0745878
total_eventos                 0.0580196
viewed product                0.0494672
condition_Bom                 0.044574
storage_16GB                  0.0414435
storage_32GB                  0.0399811
brand listing                 0.0398778
condition_Muito Bom           0.0380585
condition_Excelente           0.0372241
generic listing               0.037163
ad campaign hit               0.0343948
storage_64GB                  0.0328822
searched products             0.0298772
search engine hit             0.026603
search_engine_Google          0.0264463
channel_Paid                  0.0262191
storage_8GB                   0.0258794
visited site                  0.025552
new_vs_returning_Returning    0.0250472
storage_128GB                 0.0223949
min                           0.0211577
max                           0.0205834
channel_Direct                0.0202241
ch

In [102]:
#Hago xgboost con variables más importantes
mas_importantes = tabla.drop(["search_engine_Ask", "channel_Unknown", "new_vs_returning_New", "channel_Email", "search_engine_Yahoo", "search_engine_Bing", "storage_512MB", "lead", "channel_Social", "condition_Novo", "storage_4GB", "staticpage", "storage_256GB"],axis="columns")
mas_importantes_label = pd.merge(res,mas_importantes, on = "person", how = "inner")
mas_importantes_label.head()

,person,label,storage_128GB,storage_16GB,storage_32GB,storage_64GB,storage_8GB,condition_Bom,condition_Bom - Sem Touch ID,condition_Excelente,...,conversion,generic listing,search engine hit,searched products,viewed product,visited site,total_eventos,min,max,total_horas
0,0566e9c1,0,0.0,9.0,9.0,0.0,7.0,16.0,2.0,1.0,...,1.0,15.0,1.0,0.0,23.0,17.0,68.0,5,5,214.616111
1,6ec7ee77,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2.0,5,5,0.000000
2,abe7a2fb,0,0.0,11.0,10.0,10.0,0.0,19.0,0.0,3.0,...,0.0,9.0,4.0,6.0,31.0,22.0,96.0,2,5,2593.164444
3,34728364,0,0.0,15.0,8.0,0.0,0.0,13.0,0.0,7.0,...,0.0,3.0,1.0,0.0,24.0,4.0,37.0,5,5,206.263611
4,87ed62de,0,0.0,11.0,0.0,0.0,0.0,6.0,0.0,5.0,...,0.0,0.0,0.0,0.0,9.0,1.0,17.0,5,5,0.391389


In [103]:
y = mas_importantes_label["label"]
x = mas_importantes_label.drop(["label", "person"],axis="columns")

In [104]:
datos_train_red, datos_test_red, cat_train_red, cat_test_red = train_test_split(x, y, test_size = 0.3, random_state = 0)

In [121]:
#Hago xgboost con las variables seleccionadas
xg_reg = xgb.XGBRegressor(objective ='reg:tweedie', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 7, n_estimators = 10)

xg_reg.fit(datos_train_red,cat_train_red)

XGBRegressor(alpha=7, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='reg:tweedie', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [122]:
preds = xg_reg.predict(datos_test_red)
rmse = np.sqrt(mean_squared_error(cat_test_red, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.322143


In [123]:
a_predecir = pd.read_csv('trocafone_kaggle_test.csv')
a_predecir = pd.merge(a_predecir, mas_importantes, on = "person", how = "left")
a_predecir.head()

,person,storage_128GB,storage_16GB,storage_32GB,storage_64GB,storage_8GB,condition_Bom,condition_Bom - Sem Touch ID,condition_Excelente,condition_Muito Bom,...,conversion,generic listing,search engine hit,searched products,viewed product,visited site,total_eventos,min,max,total_horas
0,4886f805,0.0,0.0,5.0,0.0,0.0,0.0,0.0,4.0,1.0,...,0.0,1.0,1.0,1.0,4.0,1.0,9.0,5,5,0.385556
1,0297fc1e,50.0,128.0,34.0,193.0,0.0,244.0,2.0,46.0,119.0,...,0.0,21.0,0.0,6.0,404.0,95.0,567.0,1,5,3324.893889
2,2d681dd8,6.0,2.0,6.0,0.0,0.0,14.0,0.0,0.0,0.0,...,0.0,1.0,2.0,1.0,13.0,2.0,26.0,5,5,235.572222
3,cccea85e,0.0,85.0,598.0,32.0,25.0,200.0,1.0,191.0,347.0,...,0.0,20.0,26.0,1.0,739.0,22.0,836.0,5,5,568.091944
4,4c8a8b93,38.0,60.0,18.0,62.0,0.0,56.0,6.0,76.0,39.0,...,0.0,14.0,13.0,9.0,177.0,20.0,257.0,5,5,96.134444


In [124]:
personas_a_predecir["person"] = a_predecir["person"]
a_predecir = a_predecir.drop(["person"],axis="columns")
a_predecir.head()

,storage_128GB,storage_16GB,storage_32GB,storage_64GB,storage_8GB,condition_Bom,condition_Bom - Sem Touch ID,condition_Excelente,condition_Muito Bom,new_vs_returning_Returning,...,conversion,generic listing,search engine hit,searched products,viewed product,visited site,total_eventos,min,max,total_horas
0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,...,0.0,1.0,1.0,1.0,4.0,1.0,9.0,5,5,0.385556
1,50.0,128.0,34.0,193.0,0.0,244.0,2.0,46.0,119.0,94.0,...,0.0,21.0,0.0,6.0,404.0,95.0,567.0,1,5,3324.893889
2,6.0,2.0,6.0,0.0,0.0,14.0,0.0,0.0,0.0,1.0,...,0.0,1.0,2.0,1.0,13.0,2.0,26.0,5,5,235.572222
3,0.0,85.0,598.0,32.0,25.0,200.0,1.0,191.0,347.0,21.0,...,0.0,20.0,26.0,1.0,739.0,22.0,836.0,5,5,568.091944
4,38.0,60.0,18.0,62.0,0.0,56.0,6.0,76.0,39.0,19.0,...,0.0,14.0,13.0,9.0,177.0,20.0,257.0,5,5,96.134444


In [125]:
personas_a_predecir.columns = ['person']
personas_a_predecir.head()

0    4886f805
1    0297fc1e
2    2d681dd8
3    cccea85e
4    4c8a8b93
Name: person, dtype: object

In [126]:
entrega = xg_reg.predict(a_predecir)
entrega = entrega.clip(min=0)  

y_summit = pd.DataFrame(entrega)
y_summit.columns = ['label']
y_summit['person'] = personas_a_predecir
#y_summit['person'] = a_predecir.reset_index()['person']
#y_person = a_predecir.reset_index()['person']

y_summit = y_summit[['person','label']]

y_summit.to_csv('summit024.csv', index=False)